## Primary & Blooming Model

In [7]:
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16, preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.externals import joblib

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

### Load training data and labels.

In [8]:
batch_size = 32
size_img = 64
imgs = (size_img,size_img)
train_csv = data.get_class_data(train=True, label='primary')
val_csv = data.get_class_data(train=False, label='primary')
train_labels = np.asarray(train_csv)[:,1:3] 
train_data = np.asarray(train_csv)[:,0] #name of image
all_zeros_train = [index for index, label in enumerate(train_labels) if sum(label) == 0]
if(all_zeros_train > 0):
    train_data = np.delete(train_data, all_zeros_train)
    train_labels = np.delete(train_labels, all_zeros_train, axis=0)

val_labels = np.asarray(val_csv)[:,1:5]
val_data = np.asarray(val_csv)[:,0]
all_zeros_val = [index for index, label in enumerate(val_labels) if sum(label) == 0]
if(all_zeros_val > 0):
    val_data = np.delete(val_data, all_zeros_val)
    val_labels = np.delete(val_labels, all_zeros_val, axis=0)

labels=['primary', 'blooming']

#data is [image name]
print(train_labels.shape)
print(train_data.shape)
print(val_labels.shape)
print(val_data.shape)

(26284, 2)
(26284,)
(11229, 2)
(11229,)


### Generators

In [9]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels, batch_size=batch_size, img_size=imgs, balance_batches=True)

val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=batch_size, img_size=imgs, balance_batches=True)

### Network Structure

In [10]:

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(3, 64, 64)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 62, 62)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 60, 60)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 30, 30)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 30, 30)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 28, 28)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 26, 26)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 13, 13)        0         
__________

### Training Network

In [ ]:
model.fit_generator(train_gen, steps_per_epoch=len(train_data)/batch_size,
                    epochs=2, verbose=1,
                    validation_data=val_gen, validation_steps=len(val_data)/batch_size)

Epoch 1/2
